In [2]:
import pandas as pd
bad = pd.read_csv('../data/bad/bad.csv')
normal=pd.read_csv('../data/neautral/neutral.csv')

In [4]:
badwordlist = list(bad.word)#悪口単語をリストで抽出

In [116]:
'''
[形態素解析した時の単語数],[単純に悪口単語が文章に含まれているのかのフラグ], [文章中に現れる悪口数]
'''


df = pd.concat([bad, normal])

import sys
sys.path.append('/Users/nagataeiki/.pyenv/versions/3.7.4/lib/python3.7/site-packages')
import numpy as np
from janome.tokenizer import Tokenizer
import janome


#悪口が含まれている数のカウント, 単に含まれているのかのフラグを返す
def get_bad_word_count(text, badwordlist):
    textlist = []
    count=0
    flg=0
    for t in tokenizer.tokenize(text):
        if t.surface in badwordlist:
            count+=1
    for w in badwordlist:
        if w in text:
            flg=1
            return count, flg
    
    return count,flg


#dfと悪口辞書を入れて新しい特徴量を含んだdfを返す
def make_bad_feature(df, badwordlist):
    col=["content","mophologics_num", "bad_word_num", "simple_bad_flg","target"]
    basedf = pd.DataFrame(columns=col)
    tokenizer = Tokenizer()
    for r in df.iterrows():
        text = r[1].content
        target = r[1].type
        num = len(tokenizer.tokenize(text))
        count,flg = get_bad_word_count(text, badwordlist)
        
        newdf = pd.DataFrame(np.array([text, num, count,flg,target]).reshape(1,-1),columns=col)
        basedf = pd.concat([basedf, newdf])

    basedf["target"] = basedf.apply(lambda x: int(x["target"]), axis=1)
    return basedf
df =make_bad_feature(df, badwordlist)

In [123]:
df.head()

,content,mophologics_num,bad_word_num,simple_bad_flg,target
0,ガイジ,1,1,1,1
0,くどい,1,1,1,1
0,偽物 ただの釣り動画,6,2,1,1
0,振り付けダサイ,2,0,1,1
0,男きも,3,0,1,1


In [117]:
X = df[["mophologics_num", "bad_word_num", "simple_bad_flg"]]
y = df.target

In [130]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import f1_score
x_train, x_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)
svm = SVC(C=1.0, random_state=42)
svm.fit(x_train, y_train)

pred_train = svm.predict(x_train)
pred_test = svm.predict(x_test)
print(f1_score(y_train, pred_train))
print(f1_score(y_test, pred_test))

0.9731051344743276
0.9574468085106383
